In [32]:
"""
SEE: https://github.com/sudoswap/lssvm/blob/main/src/LSSVMPair.sol
Events to query for are:
  1. SwapNFTInPair()
    - emitted by LSSVMPair.swapNFTsForToken()  (sell order)
    - has no data so need to look at Transfer event for volume
  2. SwapNFTOutPair()
    - emitted by LSSVMPair.swapTokenForAnyNFTs() (buy order)
    - has no data so need to look at Transfer event for volume
  3. SpotPriceUpdate(uint128 newSpotPrice)
    - emitted by LSSVMPair on spot price changes: use to track price changes
"""

import pandas as pd
import typing as tp

from ape import accounts, chain, Contract, networks

In [33]:
networks.parse_network_choice('ethereum:mainnet:alchemy').__enter__()

<alchemy chain_id=1>

In [34]:
current_block = chain.blocks[-1].number
current_block

15461514

In [35]:
sudo_router = Contract("0x2B2e8cDA09bBA9660dCA5cB6233787738Ad68329")
sudo_router

<LSSVMRouter 0x2B2e8cDA09bBA9660dCA5cB6233787738Ad68329>

In [36]:
sudo_factory = Contract("0xb16c1342E617A5B6E4b631EB114483FDB289c0A4")
sudo_factory

<LSSVMPairFactory 0xb16c1342E617A5B6E4b631EB114483FDB289c0A4>

In [37]:
sudo_factory.NewPair

NewPair(address poolAddress)

In [38]:
# use address of top xmons pool to check is pool thru factory
sudo_factory.isPair("0x5caf332dca4e6c9e69d52f320c21e74845353db0", 0)

True

In [39]:
# utility functions to unfold event arguments into separate columns
def unfold_event_args(key: str, event_args: dict) -> tp.Any:
    return event_args[key]

def unfold(df: pd.DataFrame):
    for key in df.iloc[-1]['event_arguments'].keys():
        df[key] = df['event_arguments'].apply(lambda ev: unfold_event_args(key, ev))

In [40]:
%time query_pairs = sudo_factory.NewPair.query("*", start_block=current_block - 3000)  # last 10 hr
query_pairs

CPU times: user 276 ms, sys: 49.8 ms, total: 325 ms
Wall time: 1.48 s


,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x371C22d70C5a5623178bf7c8b40...,0xc8324193d3046fa80bb9387dbe23b2736eb9fe02e4f8...,15458578,0x460768d3eb7b2b2e34bb040e98b6646695112489a410...,191,100
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xd2a2249bc6d377257C97Bb85d83...,0x0f8f19f08ddfcd9905e268a93094cc3cf68b57ca15a7...,15458598,0x337d9e66755885d80afff2d7f3cf15a35e9d35536deb...,60,44
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x18Eeb5c34f0Db923E6768e56D45...,0x01b6093eb71f36cc9907aadd3230a39a1dc34bf27b7f...,15458635,0x5a5b20d05967319caeb66dbb6e3390a2e079b6723aa4...,34,28
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x466f7Bf00fC67b20700c4c2dd96...,0x37d4bc2cdd5f9eafc3c3777d0b92610e42c133b16439...,15458663,0xadd3bac2c01ac1b70a44ad8f8c99f364bc4325d0e8f7...,295,240
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x3708aFC92D1908a8Aaf3E7b1F92...,0x211758bbb4e22901258cab25712f8d8eb8b012a55383...,15458665,0xef6e2baee728647a01951461993eb0aa8986cff973e5...,80,36
...,...,...,...,...,...,...,...,...
97,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x35a77602a0423BBE8920d1958E6...,0x75a28f2a64280dad0bd756323a0483c50ebd7b1ef243...,15461382,0x7c9e91baedfe88c6febc874c7efd4220d92019cb923f...,98,71
98,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xb76C7a10FEAaC839f2b207AFee0...,0x2315d886c9a99ef539561ce8ad72b04f791c8c2a57f5...,15461471,0xc4412d72295dc29598736a10c74e78b8e76002a2a066...,19,21
99,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xaf26803b8361ed13eC228E7b0f2...,0x590fee25b5df700ac5728358dd57648a58ea7ae480d8...,15461497,0x910753367e6289b10bf52260fadae692f505f347b137...,133,48
100,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x065Edb5385f6b03B1035CD7efb6...,0xbb2671a39b9b2fdd4dac1dc7d21ead7fa9749e2f2e30...,15461499,0xba0b102a7e469a4476b1ce3d1027779c93f6468d274d...,81,45


In [41]:
unfold(query_pairs)

In [42]:
query_pairs

,event_name,contract_address,event_arguments,transaction_hash,block_number,block_hash,log_index,transaction_index,poolAddress
0,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x371C22d70C5a5623178bf7c8b40...,0xc8324193d3046fa80bb9387dbe23b2736eb9fe02e4f8...,15458578,0x460768d3eb7b2b2e34bb040e98b6646695112489a410...,191,100,0x371C22d70C5a5623178bf7c8b400e0643d481CcA
1,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xd2a2249bc6d377257C97Bb85d83...,0x0f8f19f08ddfcd9905e268a93094cc3cf68b57ca15a7...,15458598,0x337d9e66755885d80afff2d7f3cf15a35e9d35536deb...,60,44,0xd2a2249bc6d377257C97Bb85d83d87ddCB6bB438
2,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x18Eeb5c34f0Db923E6768e56D45...,0x01b6093eb71f36cc9907aadd3230a39a1dc34bf27b7f...,15458635,0x5a5b20d05967319caeb66dbb6e3390a2e079b6723aa4...,34,28,0x18Eeb5c34f0Db923E6768e56D45bd2eF5E33eF9D
3,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x466f7Bf00fC67b20700c4c2dd96...,0x37d4bc2cdd5f9eafc3c3777d0b92610e42c133b16439...,15458663,0xadd3bac2c01ac1b70a44ad8f8c99f364bc4325d0e8f7...,295,240,0x466f7Bf00fC67b20700c4c2dd96c89aCF1A4ffF0
4,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x3708aFC92D1908a8Aaf3E7b1F92...,0x211758bbb4e22901258cab25712f8d8eb8b012a55383...,15458665,0xef6e2baee728647a01951461993eb0aa8986cff973e5...,80,36,0x3708aFC92D1908a8Aaf3E7b1F923443AD1491114
...,...,...,...,...,...,...,...,...,...
97,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x35a77602a0423BBE8920d1958E6...,0x75a28f2a64280dad0bd756323a0483c50ebd7b1ef243...,15461382,0x7c9e91baedfe88c6febc874c7efd4220d92019cb923f...,98,71,0x35a77602a0423BBE8920d1958E6Fc412501447eA
98,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xb76C7a10FEAaC839f2b207AFee0...,0x2315d886c9a99ef539561ce8ad72b04f791c8c2a57f5...,15461471,0xc4412d72295dc29598736a10c74e78b8e76002a2a066...,19,21,0xb76C7a10FEAaC839f2b207AFee0Bd90134886AC9
99,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0xaf26803b8361ed13eC228E7b0f2...,0x590fee25b5df700ac5728358dd57648a58ea7ae480d8...,15461497,0x910753367e6289b10bf52260fadae692f505f347b137...,133,48,0xaf26803b8361ed13eC228E7b0f2b594A56f02433
100,NewPair,0xb16c1342E617A5B6E4b631EB114483FDB289c0A4,{'poolAddress': '0x065Edb5385f6b03B1035CD7efb6...,0xbb2671a39b9b2fdd4dac1dc7d21ead7fa9749e2f2e30...,15461499,0xba0b102a7e469a4476b1ce3d1027779c93f6468d274d...,81,45,0x065Edb5385f6b03B1035CD7efb6bAA3bA7de8bEe


In [43]:
# NOTE: majority of pools aren't verified with etherscan, so might cause some issues
# SEE: https://docs.apeworx.io/ape/stable/userguides/contracts.html#from-contract
Contract(query_pairs['poolAddress'].iloc[0])

ChainError: Failed to get contract type for address '0x371C22d70C5a5623178bf7c8b400e0643d481CcA'.